# Bias Experiement

In [1]:
# Data Science Packages
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from  matplotlib import pyplot
import seaborn as sns
import warnings
import copy
from sklearn.metrics import classification_report



# Seaborn Style
sns.set(style='ticks')
sns.set_style({'font.family': 'Hiragino Maru Gothic Pro'})
sns.set_palette("cool")

# Pandas Style
pd.set_option("display.max_column", 9999)
pd.set_option("display.max_row", 9999)

# Ignore annoying warning 
warnings.filterwarnings('ignore')

# Pytoch 
import torch

# custom packages
from src.datamodules.replacement_datamodule import ReplacementDataModule
from src.models.convolutional_autoencoder import ConvNetAutoEncoder

# Defining general model 

In this secction we are going to define the general model. This model is the pretrain model that later is fine tune per class (eg, `thermal-damage`, `no-hardware-issue`
)

In [6]:
from omegaconf import OmegaConf
hparams = OmegaConf.load('configs/model/conv-autoenc.yaml')
autoencoder = ConvNetAutoEncoder(lr = hparams.lr,
                                     latent_dim= hparams.latent_dim,
                                     series_dim= hparams.series_dim,
                                     features_dim= hparams.features_dim,
                                     num_category= hparams.num_category,
                                     num_numerical= hparams.num_numerical,
                                     #categorical_weight_loss = hparams.categorical_weight_loss,
                                     #numerical_weight_loss = hparams.numerical_weight_loss,
                                     num_conv_blocks= hparams.num_conv_blocks,
                                     dims_expand= hparams.dims_expand,
                                     dropout_prob= hparams.dropout_prob,
                                     weight_decay= hparams.weight_decay,
                                     inter_dim= hparams.inter_dim)
                                     #freeze_encoder = hparams.freeze_encoder,
                                     #checkpoint = hparams.checkpoint)

# Loading checkpoint autoencoder: `thermal-damage`  

Here we are going to load the weights for the autoencoder especialized in `thermal-damage`. Additionaly, we are going setup the model in evaluation mode.

In [3]:
autoencoder_thermal = copy.deepcopy(autoencoder) #copy base model
checkpoint = torch.load("logs/runs/convolutional-autoencoder-bias-thermal-damage/2021-09-17/10-12-14/checkpoints/epoch=102.ckpt")
autoencoder_thermal.load_state_dict(checkpoint['state_dict'])
autoencoder_thermal.eval()

ConvNetAutoEncoderBias(
  (encoder): ConvEncoder(
    (flatten): Flatten(start_dim=1, end_dim=-1)
    (dropout): Dropout(p=0.5, inplace=False)
    (convblocks): ModuleList(
      (0): ConvBlock(
        (conv1d): Conv1d(160, 320, kernel_size=(3,), stride=(1,), padding=(1,))
        (batchnorm1d): BatchNorm1d(320, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (maxpool1d): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (1): ConvBlock(
        (conv1d): Conv1d(320, 640, kernel_size=(3,), stride=(1,), padding=(1,))
        (batchnorm1d): BatchNorm1d(640, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (maxpool1d): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
    )
    (linear): Linear(in_features=5120, out_features=250, bias=True)
  )
  (decoder): ConvDecoder(
    (inverse_convblocks): ModuleList(
      (0): InverseConvBlock(
        (conv1d_trans): ConvTranspos

# Loading AutoEncoder for W724Ci family

In [7]:
autoencoder_w724 = copy.deepcopy(autoencoder) #copy base model
checkpoint = torch.load("logs/runs/convolutional-autoencoder/2021-09-28/10-57-37/checkpoints/epoch=85.ckpt")
autoencoder_w724.load_state_dict(checkpoint['state_dict'])
autoencoder_w724.eval()

ConvNetAutoEncoder(
  (encoder): ConvEncoder(
    (flatten): Flatten(start_dim=1, end_dim=-1)
    (dropout): Dropout(p=0, inplace=False)
    (convblocks): ModuleList(
      (0): ConvBlock(
        (conv1d): Conv1d(160, 320, kernel_size=(3,), stride=(1,), padding=(1,))
        (batchnorm1d): BatchNorm1d(320, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (maxpool1d): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (1): ConvBlock(
        (conv1d): Conv1d(320, 640, kernel_size=(3,), stride=(1,), padding=(1,))
        (batchnorm1d): BatchNorm1d(640, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (maxpool1d): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
    )
    (linear_2): Linear(in_features=5120, out_features=1024, bias=True)
    (linear): Linear(in_features=1024, out_features=256, bias=True)
  )
  (decoder): ConvDecoder(
    (inverse_convblocks): ModuleList(


# Define Dataloader

Here we are going to define the data loader. In the Notebook always `num_workers` must be zero.

In [11]:
hparams_datamodule = OmegaConf.load("configs/datamodule/cpe-replacement.yaml")
datamodule = ReplacementDataModule(data_dir = "data/preprocessed/data_w32_g3/Family-All/maxmin",
                                     train_val_split = 0.9,
                                     batch_size = 5000000,# Loss comparisson chancge teh BS to 5000000
                                     num_workers = 0,
                                     pin_memory = False,
                                     features_dim = 160,
                                     series_dim = 32,
                                     name_labels = ["label_minor_physical_thermal_damage"],
                                     sampler=True,
                                     weights_minority_class = 8,
                                     features_numerical= hparams_datamodule.features_numerical,
                                     features_categorical= hparams_datamodule.features_categorical)

Now we are just to init the test stage of the datamodule.

In [12]:
datamodule.setup(stage='fit')
train_dataloader = datamodule.train_dataloader()
val_dataloader = datamodule.val_dataloader()

Samples weights for Balance Random Sampler: 
    synthetic_class  indices    weights
0              0.0     6219   1.049043
1              1.0      305  21.390164
Samples weights for Custom Random Sampler: 
    synthetic_class  weights
0              0.0      1.0
1              1.0      8.0


And we can access to a batch in the data set as follows

In [14]:
dataiter = iter(train_dataloader)
input, labels = dataiter.next()

In [15]:
input.shape

torch.Size([6524, 32, 160])

In [16]:
autoencoder_w724(input).shape

torch.Size([6524, 32, 160])

# Approach 1: Compare only the reconsturction loss of `cpe_type_W724Ci`

The only thing we need to compres is the input with the reconstruction and visualized the losses.

In [17]:
criterion_mse = torch.nn.MSELoss(reduce=False)
reconstruction = autoencoder_w724(input)
mse_loss =criterion_mse(reconstruction,input).mean(dim=(1,2))

Now we can compare the reconstruction loss with a threshold to obtain the predictions.

In [18]:
mse_loss

tensor([0.0003, 0.0007, 0.0008,  ..., 0.0005, 0.0012, 0.0019],
       grad_fn=<MeanBackward1>)

In [33]:
mse_loss.shape

torch.Size([6524])

In [34]:
torch.min(mse_loss)

tensor(0.0002, grad_fn=<MinBackward1>)

In [35]:
torch.max(mse_loss)

tensor(0.0216, grad_fn=<MaxBackward1>)

In [30]:
torch.mean(mse_loss)

tensor(0.0013, grad_fn=<MeanBackward0>)

In [48]:
threshold =  0.0013
preds = (mse_loss < threshold).float() * 1

Here is the classification report 

In [49]:
print(classification_report(labels, preds))

              precision    recall  f1-score   support

         0.0       0.88      0.42      0.57      4625
         1.0       0.38      0.85      0.52      1899

    accuracy                           0.55      6524
   macro avg       0.63      0.64      0.55      6524
weighted avg       0.73      0.55      0.56      6524



21/09/29 14:29:56 WARN TransportChannelHandler: Exception in connection from /192.168.0.101:56165
java.io.IOException: Operation timed out
	at sun.nio.ch.FileDispatcherImpl.read0(Native Method)
	at sun.nio.ch.SocketDispatcher.read(SocketDispatcher.java:39)
	at sun.nio.ch.IOUtil.readIntoNativeBuffer(IOUtil.java:223)
	at sun.nio.ch.IOUtil.read(IOUtil.java:192)
	at sun.nio.ch.SocketChannelImpl.read(SocketChannelImpl.java:378)
	at io.netty.buffer.PooledByteBuf.setBytes(PooledByteBuf.java:253)
	at io.netty.buffer.AbstractByteBuf.writeBytes(AbstractByteBuf.java:1133)
	at io.netty.channel.socket.nio.NioSocketChannel.doReadBytes(NioSocketChannel.java:350)
	at io.netty.channel.nio.AbstractNioByteChannel$NioByteUnsafe.read(AbstractNioByteChannel.java:148)
	at io.netty.channel.nio.NioEventLoop.processSelectedKey(NioEventLoop.java:714)
	at io.netty.channel.nio.NioEventLoop.processSelectedKeysOptimized(NioEventLoop.java:650)
	at io.netty.channel.nio.NioEventLoop.processSelectedKeys(NioEventLoop.jav

# Approach 2: Siamese Network Loss

In the previous approach can be difficul for the model to compare just againt a threshold due to the fact that a sample can be just hard to reconstruct independly of the class. Therefore, in this seccion we are going to use a Siamese Network one finetune on `themal-damage` and the other one `all classes`. 

In [30]:
autoencoder_base = copy.deepcopy(autoencoder) #copy base model
checkpoint = torch.load("logs/runs/convolutional-autoencoder-bias-no-problem/2021-09-17/16-54-44/checkpoints/epoch=116.ckpt")
autoencoder_base.load_state_dict(checkpoint['state_dict'])
autoencoder_base.eval()

ConvNetAutoEncoderBias(
  (encoder): ConvEncoder(
    (flatten): Flatten(start_dim=1, end_dim=-1)
    (dropout): Dropout(p=0.5, inplace=False)
    (convblocks): ModuleList(
      (0): ConvBlock(
        (conv1d): Conv1d(160, 320, kernel_size=(3,), stride=(1,), padding=(1,))
        (batchnorm1d): BatchNorm1d(320, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (maxpool1d): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (1): ConvBlock(
        (conv1d): Conv1d(320, 640, kernel_size=(3,), stride=(1,), padding=(1,))
        (batchnorm1d): BatchNorm1d(640, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (maxpool1d): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
    )
    (linear): Linear(in_features=5120, out_features=250, bias=True)
  )
  (decoder): ConvDecoder(
    (inverse_convblocks): ModuleList(
      (0): InverseConvBlock(
        (conv1d_trans): ConvTranspos

In [31]:
criterion_mse = torch.nn.MSELoss(reduce=False)

# Thermal damage
reconstruction = autoencoder_thermal(input)
mse_loss_thermal =criterion_mse(reconstruction,input).mean(dim=(1,2))

# Base
reconstruction = autoencoder_base(input)
mse_loss_base =criterion_mse(reconstruction,input).mean(dim=(1,2))

In [52]:
threshold_thermal = 0.013
preds_thermal = (threshold_thermal > mse_loss_thermal).float() * 1#
preds_thermal[:20]

tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1.,
        0., 0.])

In [59]:
threshold_noissue = 0.00001
preds_hard = (threshold_noissue < mse_loss_thermal).float() * 1
preds_hard[:20]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.])

In [60]:
(preds_hard*preds_thermal)[:20]

tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1.,
        0., 0.])

In [61]:
print(classification_report(labels, preds_hard*preds_thermal))

              precision    recall  f1-score   support

         0.0       0.98      0.83      0.90      2060
         1.0       0.43      0.89      0.58       305

    accuracy                           0.83      2365
   macro avg       0.71      0.86      0.74      2365
weighted avg       0.91      0.83      0.86      2365



# Approach 3: Siamese Network Classifier

In this section we are going to use 2 embedding (general and  themal damage) as feature extactor to traing a classifier.

In [5]:
autoencoder_thermal = copy.deepcopy(autoencoder) #copy base model
checkpoint = torch.load("logs/runs/convolutional-autoencoder-bias-thermal-damage/2021-09-17/10-12-14/checkpoints/epoch=102.ckpt")
autoencoder_thermal.load_state_dict(checkpoint['state_dict'])


autoencoder_base = copy.deepcopy(autoencoder) #copy base model
checkpoint = torch.load("logs/runs/convolutional-autoencoder/2021-09-17/01-26-12/checkpoints/epoch=40.ckpt")
autoencoder_base.load_state_dict(checkpoint['state_dict'])

autoencoder_noissue = copy.deepcopy(autoencoder) #copy base model
checkpoint = torch.load("logs/runs/convolutional-autoencoder-bias-no-problem/2021-09-17/16-54-44/checkpoints/epoch=116.ckpt")
autoencoder_noissue.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

Let froze the paramaters and define the encoders

In [6]:
autoencoder_thermal.freeze()
encoder_thermal = autoencoder_thermal.encoder

#autoencoder_base.freeze()
encoder_base = autoencoder_base.encoder

autoencoder_noissue.freeze()
encoder_noissue = autoencoder_noissue.encoder

In [7]:
import pytorch_lightning as pl
from pytorch_lightning import LightningDataModule, LightningModule, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import WandbLogger


import torch
import torch.nn as nn
import torchmetrics

class SiameseClassifier(pl.LightningModule):
    """
    Convolutional Autoencoder.
    """
    def __init__(self,
                 lr = 0.0003,
                 dropout_prob= 0.25,
                 weight_decay= 0.05,
                 latent_dim = 750,
                 inter_dim = 64,
                 output_dim = 1,
                 freeze_encoder = True,
                 encoder_thermal= autoencoder_thermal,
                 encoder_base= autoencoder_base,
                 name_labels_logger=['label_minor_physical_thermal_damage'],
                 **kwargs):
       
        super(SiameseClassifier, self).__init__()

        self.save_hyperparameters()

        self.dropout = nn.Dropout(p=self.hparams.dropout_prob)

        self.input_linear = nn.Linear(self.hparams.latent_dim,
                                      self.hparams.inter_dim)

        self.output_linear = nn.Linear(self.hparams.inter_dim,
                                       self.hparams.output_dim)


        # Freeze Pretrain encoder
        #if self.hparams.freeze_encoder:
        #    autoencoder_thermal.freeze()
        #    autoencoder_base.freeze()
            
        self.encoder_thermal = autoencoder_thermal.encoder
        self.encoder_base = autoencoder_base.encoder
        self.encoder_noissue = autoencoder_noissue.encoder
        

        # loss function
        self.criterion = torch.nn.BCELoss()

    def forward(self, x):
        # get embeddings
        embeddings_thermal = self.encoder_thermal(x)
        embeddings_base = self.encoder_base(x)
        embeddings_noissue = self.encoder_noissue(x)
        
        #concat embeddings
        embeddings = torch.cat([embeddings_base, embeddings_thermal,embeddings_noissue], dim=1)
        #classifier
        x = torch.relu((self.dropout(self.input_linear(embeddings))))
        x = torch.sigmoid((self.output_linear(x)))
        return x

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(),
                                 lr=self.hparams.lr,
                                 weight_decay=self.hparams.weight_decay)
    
    def metrics_logger_custom(self, predictions, targets, prefix):
        prefix = prefix + '/'
        outputs = dict()
        for idx, l in enumerate(self.hparams.name_labels_logger):
            preds = predictions[:, idx]
            target = targets[:, idx].int()
            outputs[prefix + l + '/accuracy'] = torchmetrics.functional.accuracy(preds, target)
            outputs[prefix + l + '/precision'] = torchmetrics.functional.precision(preds, target)
            outputs[prefix + l + '/recall'] = torchmetrics.functional.recall(preds, target)
            outputs[prefix + l + '/f1'] = torchmetrics.functional.f1(preds, target)
        return outputs
    
    def training_step(self, batch, batch_idx):
        input, labels  = batch
        predictions = self(input)
        loss = self.criterion(predictions, labels)
        outputs = self.metrics_logger_custom(predictions, labels, 'train')
        self.log('train/loss_step', loss)
        return {'loss': loss, 'metrics': outputs}

    def training_epoch_end(self, outputs):
        for _, metric in enumerate(outputs[0]['metrics'].keys()):
            self.log(metric, torch.tensor([x['metrics'][metric] for x in outputs]).mean())

    def validation_step(self, batch, batch_idx):
        input, labels = batch
        predictions = self(input)
        loss = self.criterion(predictions, labels)
        outputs = self.metrics_logger_custom(predictions, labels, 'val')
        outputs['val/loss'] = loss
        return outputs

    def validation_epoch_end(self, outputs):
        for _, metric in enumerate(outputs[0].keys()):
            self.log(metric, torch.tensor([x[metric] for x in outputs]).mean())

    def test_step(self, batch, batch_idx):
        input, labels = batch
        predictions = self(input)
        outputs = self.metrics_logger_custom(predictions, labels, 'test')
        return outputs

    def test_epoch_end(self, outputs):
        for _, metric in enumerate(outputs[0].keys()):
            self.log(metric, torch.tensor([x[metric] for x in outputs]).mean())


In [10]:
model = SiameseClassifier()

wandb_logger = WandbLogger(name='siamese_classifer_v4',project='siamese_classifer')

trainer = Trainer(max_epochs=100, logger= wandb_logger, callbacks=[EarlyStopping(monitor="val/loss")])
trainer.fit(model, train_dataloader, val_dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
wandb: Currently logged in as: felipevilla (use `wandb login --relogin` to force relogin)



  | Name            | Type        | Params
------------------------------------------------
0 | dropout         | Dropout     | 0     
1 | input_linear    | Linear      | 48.1 K
2 | output_linear   | Linear      | 65    
3 | encoder_thermal | ConvEncoder | 2.1 M 
4 | encoder_base    | ConvEncoder | 2.1 M 
5 | encoder_noissue | ConvEncoder | 2.1 M 
6 | criterion       | BCELoss     | 0     
------------------------------------------------
2.1 M     Trainable params
4.1 M     Non-trainable params
6.2 M     Total params
24.806    Total estimated model params size (MB)


Epoch 0:  90%|█████████ | 102/113 [09:52<01:03,  5.81s/it, loss=0.207, v_num=2wjv]
Validating: 0it [00:00, ?it/s]
Epoch 1:  90%|█████████ | 102/113 [08:08<00:52,  4.79s/it, loss=0.187, v_num=2wjv]
Validating: 0it [00:00, ?it/s]
Epoch 2:  91%|█████████ | 103/113 [05:41<00:33,  3.31s/it, loss=0.167, v_num=2wjv]
Validating: 0it [00:00, ?it/s]
Epoch 3:  90%|█████████ | 102/113 [06:57<00:44,  4.09s/it, loss=0.162, v_num=2wjv]
Validating: 0it [00:00, ?it/s]
Epoch 3: 100%|██████████| 113/113 [07:08<00:00,  3.79s/it, loss=0.162, v_num=2wjv]
